In [7]:
import os
import pandas as pd
import numpy as np
import timeit


# Functions

In [8]:
# def get_junction_coordinates(df, coordinates_col, sep=':', shift_to_1based=True):
#     df['strand'] = None
#     df['junction_coordinate'] = None

#     for idx, row in df.iterrows():
#         kmer_coordinates = [int(x) for x in row[coordinates_col].split(sep) if x !='None']

#         if kmer_coordinates[1] < kmer_coordinates[2]: # order strand +

#             df.loc[idx, 'strand'] = '+'
#             if len(kmer_coordinates) == 4:  # 2 exons
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:3]])
#             elif len(kmer_coordinates) == 6:
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:5]])
#         else: # order strand +
#             df.loc[idx, 'strand'] = '-'
#             if len(kmer_coordinates) == 4:  # 2 exons
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
#                                                                                 kmer_coordinates[0]]])
#             elif len(kmer_coordinates) == 6:
#                 df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
#                                                                                 kmer_coordinates[0],
#                                                                                 kmer_coordinates[2],
#                                                                                 kmer_coordinates[5]
#                                                                                ]])
#     return df




In [9]:
def explode_immunopepper_coord2(mx, coord_col = 'coord', sep=':'):
    
    coord_mx = mx[coord_col].str.split(sep, expand=True) #7 min
    
    dummy_fill = -9999
    coord_mx = coord_mx.astype(float)
    coord_mx.fillna(dummy_fill, inplace=True)
    coord_mx = coord_mx.astype(int)

    coord_mx['strand'] = None
    coord_mx.loc[coord_mx[1] < coord_mx[2], 'strand'] = '+'
    coord_mx.loc[coord_mx[1] > coord_mx[2], 'strand'] = '-'

    coord_mx['junction_coordinate1'] = None
    coord_mx['junction_coordinate2'] = None

    if 4 not in coord_mx.columns:
        coord_mx[4] = dummy_fill
        coord_mx[5] = dummy_fill
    
    coord_mx = coord_mx.astype(str) # 7 min
    
    coord_mx['+first'] = coord_mx[1] + ':' + coord_mx[2]
    coord_mx['+secon'] = coord_mx[3] + ':' + coord_mx[4]
    coord_mx['-first'] = coord_mx[3] + ':' + coord_mx[0]
    coord_mx['-secon'] = coord_mx[5] + ':' + coord_mx[2]
    
        
    coord_mx[4] = ['None' if str(dummy_fill) in jx2 else jx2 for jx2 in coord_mx[4]]
    coord_mx[5] = ['None' if str(dummy_fill) in jx2 else jx2 for jx2 in coord_mx[5]]

    coord_mx.loc[(coord_mx['strand'] == '+'), 'junction_coordinate1'] = coord_mx['+first'] 
    coord_mx.loc[(coord_mx['strand'] == '-'), 'junction_coordinate1'] = coord_mx['-first'] 
    coord_mx.loc[(coord_mx['strand'] == '+') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['+secon']
    coord_mx.loc[(coord_mx['strand'] == '-') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['-secon']
    
    return coord_mx

In [10]:
def explode_immunopepper_coord(mx, coord_col = 'coord', sep=':'):
    coord_mx = mx[coord_col].str.split(sep, expand=True) #7 min

    coord_mx[1] = coord_mx[1].astype('int')
    coord_mx[2] = coord_mx[2].astype('int')

    coord_mx['strand'] = None
    coord_mx.loc[coord_mx[1] < coord_mx[2], 'strand'] = '+'
    coord_mx.loc[coord_mx[1] > coord_mx[2], 'strand'] = '-'

    coord_mx['junction_coordinate1'] = None
    coord_mx['junction_coordinate2'] = None

    
    coord_mx = coord_mx.astype(str) # 7 min
    
    coord_mx['+first'] = coord_mx[1] + ':' + coord_mx[2]
    coord_mx['+secon'] = coord_mx[3] + ':' + coord_mx[4]
    coord_mx['-first'] = coord_mx[3] + ':' + coord_mx[0]
    coord_mx['-secon'] = coord_mx[5] + ':' + coord_mx[2]
    


    coord_mx.loc[(coord_mx['strand'] == '+'), 'junction_coordinate1'] = coord_mx['+first'] 
    coord_mx.loc[(coord_mx['strand'] == '-'), 'junction_coordinate1'] = coord_mx['-first'] 
    coord_mx.loc[(coord_mx['strand'] == '+') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['+secon']
    coord_mx.loc[(coord_mx['strand'] == '-') & (coord_mx[4] != 'None') , 'junction_coordinate2'] = coord_mx['-secon']
    
    return coord_mx

# Main

In [11]:
# Paths
basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'

intermediate_file = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_order_r_complete.tsv.gz')

longlist = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/All_samples_longlist.tsv.gz'



In [12]:
# Load
inter = pd.read_csv(intermediate_file, sep = '\t')
long_list = pd.read_csv(longlist, sep = '\t', header = None)
long_list.columns = ['kmer', 'coord']

In [13]:
display(inter.head()) 
inter.shape

display(long_list.head()) 
long_list.shape

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=2.0,cancerCohortfilter >=3.0,cancerCohortfilter >=5.0,cancerCohortfilter >=10.0,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated
0,RKSTQMPCT,1218.0,689.0,249.0,89.0,26.0,4.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
1,RKSTQMPCT,1279.0,274.0,70.0,28.0,5.0,1.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
2,RKSTQMPCT,7350.0,4622.0,2653.0,1745.0,1126.0,734.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
3,RKSTQMPCT,2025.0,479.0,96.0,35.0,8.0,1.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN
4,RKSTQMPCT,8641.0,6957.0,4789.0,3211.0,1347.0,191.0,92379857:92379859:92611313:92611338:None:None,True,False,...,372,367,352,295,20.982616,8.422932,17.304198,16.069005,19.585264,NaN


,kmer,coord
0,AAAAAADPT,6415704:6415729:6415616:6415618:None:None
1,AAAAAAFAR,54284699:54284716:54284521:54284531:None:None
2,AAAAAAFAR,54285554:54285565:54284521:54284537:None:None
3,AAAAAAHHS,1254289:1254303:1254422:1254435:None:None
4,AAAAAAPTY,2463179:2463197:2463111:2463120:None:None


(393819, 2)

In [15]:
# Merge on metadata 
batch_interest = inter.merge(long_list, on = ['kmer', 'coord'], how = 'right')

print(batch_interest.shape)

print(len(batch_interest['batch'].unique()))

(431537, 24)
5838


In [16]:
# Add Coordinate
coord_mx = explode_immunopepper_coord(batch_interest)

batch_interest = pd.concat([batch_interest, 
                            coord_mx[['strand', 'junction_coordinate1', 'junction_coordinate2']]], axis = 1)

In [18]:
batch_interest.shape

(431537, 27)

In [17]:
batch_interest.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=10.0,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate1,junction_coordinate2
0,AAAAAADPT,8.0,6.0,2.0,0.0,0.0,0.0,6415704:6415729:6415616:6415618:None:None,False,False,...,0,0.839305,0.0,0.000000,0.0,0.00000,NaN,-,6415618:6415704,None
1,AAAAAAFAR,NaN,NaN,NaN,NaN,NaN,NaN,54284699:54284716:54284521:54284531:None:None,False,False,...,0,0.000000,0.0,0.961344,0.0,0.00000,NaN,-,54284531:54284699,None
2,AAAAAAFAR,NaN,NaN,NaN,NaN,NaN,NaN,54285554:54285565:54284521:54284537:None:None,False,False,...,0,0.000000,0.0,0.000000,0.0,1.45076,NaN,-,54284537:54285554,None
3,AAAAAAHHS,NaN,NaN,NaN,NaN,NaN,NaN,1254289:1254303:1254422:1254435:None:None,False,False,...,0,0.000000,0.0,0.961344,0.0,0.00000,NaN,+,1254303:1254422,None
4,AAAAAAPTY,NaN,NaN,NaN,NaN,NaN,NaN,2463179:2463197:2463111:2463120:None:None,False,False,...,0,0.000000,0.0,1.922689,0.0,0.00000,NaN,-,2463120:2463179,None


In [26]:
# Merge peptides 
debut_time = timeit.default_timer()
tot_shape = 0
extracted_metadata = []
for idx, gene in enumerate(batch_interest['batch'].unique()):
    # Load metadata
    base_meta = os.path.join(basedir, 'cohort_mutNone', f'tmp_out_ref_batch_{gene}', 'ref_sample_peptides_meta.gz')
    meta = pd.read_csv(base_meta, sep = '\t') # Note they are peptides 'NA' confused with Nan 
    meta = meta[~meta['peptide'].isna()]
    meta = meta.set_index(np.arange(len(meta)))
    meta_coord = explode_immunopepper_coord2(meta, coord_col='modifiedExonsCoord', sep=';')
    meta = pd.concat([meta, meta_coord[['strand', 'junction_coordinate1', 'junction_coordinate2']]], axis = 1)
    
    # Merge with coordinates interest
    batch_col = ['coord', 'kmer', 'junction_coordinate1', 'junction_coordinate2']
    selected_pep = batch_interest.loc[:, batch_col].merge(meta, 
                                        on = ['junction_coordinate1', 'junction_coordinate2'], 
                                        how = 'inner')
    # Restrict merge on kmer presence
    selected_pep['keep'] = [selected_pep.loc[ix, 'kmer'] in pep 
                            for ix, pep in enumerate(selected_pep['peptide'])]
    selected_pep = selected_pep.loc[selected_pep['keep'] == True]
    
    # Prepare save
    extracted_metadata.append(selected_pep)
    tot_shape+=selected_pep.shape[0]
    
    if idx % 100 == 0:
        print(f'....{idx} gene number {gene} total peptides {tot_shape} add {selected_pep.shape[0]}')
        print
print(timeit.default_timer() - debut_time)
print(tot_shape)



....0 gene number 27329 total peptides 5263 add 5263
....100 gene number 12422 total peptides 731739 add 30447
....200 gene number 27996 total peptides 985198 add 529
....300 gene number 27294 total peptides 1485466 add 205
....400 gene number 57473 total peptides 1857163 add 1434
....500 gene number 12925 total peptides 2121413 add 50
....600 gene number 58204 total peptides 2254176 add 1002
....700 gene number 54115 total peptides 2327925 add 8
....800 gene number 57182 total peptides 2363795 add 334
....900 gene number 55141 total peptides 2494529 add 230
....1000 gene number 9288 total peptides 2533307 add 214
....1100 gene number 27576 total peptides 2608003 add 355
....1200 gene number 28733 total peptides 2650252 add 48
....1300 gene number 23713 total peptides 2698564 add 210
....1400 gene number 1484 total peptides 2722764 add 59
....1500 gene number 46525 total peptides 2756946 add 87
....1600 gene number 19118 total peptides 2784196 add 214
....1700 gene number 55754 total p

In [28]:
len(extracted_metadata)

5838

In [27]:
import pickle

In [29]:
file_ = open('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/peptides.pickle', 'wb')

# dump information to that file
pickle.dump(extracted_metadata, file_)

In [31]:
res = pd.concat(extracted_metadata, axis = 0)

In [34]:
res.to_csv('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/peptides.tsv.gz', 
          sep = '\t', 
          index= None, 
          compression = 'gzip')

In [43]:
meta.loc[meta['id'] == 'ENSG00000167996.16:1775_1290:0:61965419:2-exons', 'peptide'].isna()

18378    True
Name: peptide, dtype: bool

# TESTS

In [122]:
# base_meta = os.path.join(basedir, 'cohort_mutNone', f'tmp_out_ref_batch_{21722}', 'ref_sample_peptides_meta.gz')
# meta = pd.read_csv(base_meta, sep = '\t')
# meta_coord = explode_immunopepper_coord2(meta, coord_col='modifiedExonsCoord', sep=';')
# #meta_coord2 = get_junction_coordinates(meta, 'modifiedExonsCoord', ';')
# meta = pd.concat([meta, meta_coord[['strand', 'junction_coordinate1', 'junction_coordinate2']]], axis = 1)


In [123]:
kmer = 'AAAAAAFAR'

sum([kmer in m for m in meta['peptide']])

[ix for ix, m in enumerate(meta['peptide']) if kmer in m ]

[23, 30, 87, 207, 420, 423]

In [124]:
meta.loc[np.array([23, 30, 87, 207, 420, 423]), 'modifiedExonsCoord']

23     54285554;54285613;54284496;54284537
30     54284699;54285613;54284496;54284531
87     54284701;54285613;54284119;54284530
207    54284701;54285613;54284496;54284530
420    54284699;54285613;54284119;54284531
423    54285554;54285613;54284119;54284537
Name: modifiedExonsCoord, dtype: object

In [125]:
meta['junction_coordinate2']

0         -9999:54284701
1         -9999:54285397
2      54285613:54285783
3         -9999:54284645
4      54284312:54284496
             ...        
582       -9999:54285507
583       -9999:54284119
584       -9999:54283303
585       -9999:54285064
586       -9999:54284119
Name: junction_coordinate2, Length: 587, dtype: object

['None',
 'None',
 '54285613:54285783',
 'None',
 '54284312:54284496',
 'None',
 'None',
 'None',
 'None',
 '54285196:54285595',
 'None',
 '54284312:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54284312:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54283936:54284496',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54285613:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54285613:54285783',
 '54284312:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54284312:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54285613:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54285613:54285783',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '54285196:54285595',
 'N

In [115]:
meta.loc[420, ['junction_coordinate1', 'junction_coordinate2']]

junction_coordinate1    54284531:54284699
junction_coordinate2       -9999:54284119
Name: 420, dtype: object

In [108]:
batch_interest.loc[1]

kmer                                                             AAAAAAFAR
gtexCohortfilter >0.0                                                  NaN
gtexCohortfilter >=1.0                                                 NaN
gtexCohortfilter >=2.0                                                 NaN
gtexCohortfilter >=3.0                                                 NaN
gtexCohortfilter >=5.0                                                 NaN
gtexCohortfilter >=10.0                                                NaN
coord                        54284699:54284716:54284521:54284531:None:None
junctionAnnotated                                                    False
readFrameAnnotated                                                   False
isCrossJunction                                                       True
batch                                                                21722
cancerCohortfilter >0.0                                                  8
cancerCohortfilter >=1.0 

In [107]:
meta.loc[23]

peptide                            GAQYELKDSPGVQHPAAAAAAAFARPAEPEGGT
id                      ENSG00000177508.12:143_81:0:54285613:2-exons
readFrame                                                          2
readFrameAnnotated                                             False
geneName                                          ENSG00000177508.12
geneChr                                                        chr16
geneStrand                                                         -
mutationMode                                                     ref
junctionAnnotated                                                  0
hasStopCodon                                                     NaN
isInJunctionList                                                   0
isIsolated                                                       NaN
variantComb                                                      NaN
variantSegExpr                   54285554;54285613;54284497;54284537
modifiedExonsCoord               5

In [96]:
batch_interest.loc[batch_interest['batch'] == 21722, 'kmer']

1         AAAAAAFAR
2         AAAAAAFAR
244       AAAFPHPHL
1332      AAFPHPHLL
4158      AAVAPSAFA
8837      AFCPSAPAG
9198      AFPHPHLLG
9567      AFYPYGQRS
22301     APSAFARPA
32105     AVAPSAFAR
76130     EKIMLGLPG
91222     FCPSAPAGT
97266     FPGPPPGCR
97310     FPHPHLLGL
97311     FPHPHPAFC
102163    FYPYGQRSP
102660    GAAVAPSAF
108215    GEKIMLGLP
121853    GPPPGCRRR
140854    HPAFCPSAP
141441    HPHLLGLPG
141444    HPHPAFCPS
154081    IMLGLPGAA
166198    KGEKIMLGL
167967    KIMLGLPGA
211819    LSPAAAAAF
222623    MLGLPGAAG
240307    PAFCPSAPA
240383    PAFYPYGQR
247972    PFPGPPPGC
250503    PGPPPGCRR
252395    PHPAFCPSA
252470    PHPHLLGLP
252471    PHPHPAFCP
261175    PPPGCRRRL
268362    PSAFARPAE
276792    PYGQRSPKA
330144    SAFARPAEP
347705    SPAAAAAFA
375396    TKGEKIMLG
392649    VAPSAFARP
425102    YGQRSPKAE
428422    YPYGQRSPK
Name: kmer, dtype: object

In [77]:
batch_interest.shape

(431537, 27)

In [78]:
meta.shape

(616, 23)

[False, True, False, True, False, True]

'SRSRSRHSRKSRSRSGSSKSSHSKSRSRSRSGSRSRSRSRSRSRSKSKDKRKGRKRSREESRSRSRSRSKSERSRKRGSKRDSKAGSSKKKKKEDTDRSQSRSPSRSVSKEREHAKSESSQREGRGESENAGTNQETRSRSRSNSKSKPNLPSESRSRSKSASKTRSRSKSRSRSASRSPSRSRSRSHSRS'

In [83]:
foo['peptide']

0    GRAPAPGAG
1    GRAPAPGAG
2    GRAPAPGAG
3    GRAPAPGAG
4    GRAPAPGAG
5    GRAPAPGAG
Name: kmer, dtype: object

In [73]:
meta.head()


,peptide,id,readFrame,readFrameAnnotated,geneName,geneChr,geneStrand,mutationMode,junctionAnnotated,hasStopCodon,...,variantSegExpr,modifiedExonsCoord,originalExonsCoord,vertexIdx,junctionExpr,segmentExpr,kmerType,strand,junction_coordinate1,junction_coordinate2
0,LALEADIPVRAEAEVAAAKAVILRVDLGPGRAPAPGARAGAGARVG...,ENSG00000116350.17:168_145:0:29150190:2-exons,1,False,ENSG00000116350.17,chr1,-,ref,0,NaN,...,29150102;29150190;29149098;29149226,29150102;29150192;29149098;29149226,168;145,2-exons,NaN,NaN,NaN,-,29149226:29150102,-9999:29149098
1,SGSRSRSKSRSRSQSRSRSKKEKSRSPSKEKSRSRSHSAGKSRSKR...,ENSG00000116350.17:145_12:0:29149225:2-exons,1,False,ENSG00000116350.17,chr1,-,ref,1,NaN,...,29149098;29149225;29147743;29148753,29149098;29149226;29147742;29148753,145;12,2-exons,NaN,NaN,NaN,-,29148753:29149098,-9999:29147742
2,LLCSEPTHVFGRTRWLTLANGNQKTSESNLVVTYPQLRGASPCPKV...,ENSG00000116350.17:251_245:0:29168637:2-exons,0,False,ENSG00000116350.17,chr1,-,ref,0,NaN,...,29168502;29168637;29166729;29166912,29168502;29168638;29166728;29166912,251;245,2-exons,NaN,NaN,NaN,-,29166912:29168502,-9999:29166728
3,RAPAPGARAGAGARVGAGARKRKAGAPAVAAAARVRGAESEAASET...,ENSG00000116350.17:149_15:0:29149224:2-exons,0,False,ENSG00000116350.17,chr1,-,ref,1,NaN,...,29149140;29149224;29147742;29148792,29149140;29149226;29147742;29148792,149;15,2-exons,NaN,NaN,NaN,-,29148792:29149140,-9999:29147742
4,SGSRSRSKSRSRSQSRSRSKKEKSRSRSKSKDKRKGRKRSREESRS...,ENSG00000116350.17:155_112:0:29149225:2-exons,2,False,ENSG00000116350.17,chr1,-,ref,0,NaN,...,29149157;29149225;29148670;29148866,29149157;29149226;29148670;29148866,155;112,2-exons,NaN,NaN,NaN,-,29148866:29149157,-9999:29148670


In [44]:
float('nan')

nan

In [20]:
# with open(os.path.join(basedir, 'batches_filters_19_May.tsv'), 'w') as fp:
#     for batch in batch_interest['batch'].unique():
#         fp.write(f'cohort_mutNone/tmp_out_ref_batch_{batch}/ref_sample_peptides_meta.gz\n')
